<h3><span style="font-weight:bold"> 포스코홀딩스 주식정보 가져오기 </span></h3>

In [ ]:
# 모듈 불러오기
from datetime import datetime
from datetime import timedelta
import requests
import pandas as pd
from bs4 import BeautifulSoup




In [1]:
from datetime import datetime
from datetime import timedelta
import requests
import pandas as pd
from bs4 import BeautifulSoup

stock_code = "005490"
start_ymd = 20220406
end_ymd = 20220406
df = pd.DataFrame(columns=["ymd", "time", "price"])

host = "https://finance.naver.com"
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
}
# SSL 경고창 없애기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

if str(start_ymd)[4:6] != str(end_ymd)[4:6]:
    print("시작일과 종료일의 달(month)은 같아야 합니다.")
else:
    for ymd in range(start_ymd, end_ymd+1):
        print(str(ymd) + " : 수집중...")
        url = host+"/item/sise_time.nhn?code="+stock_code+"&thistime="+str(ymd)+"161036&page=1"
        request_results = requests.get(url, headers=header, verify=False)
        soup = BeautifulSoup(request_results.text, "html.parser")

        page_url = host+"/item/sise_time.nhn?code=005490&thistime="+str(ymd)+"161036&page="
        last_page_no = int(soup.findAll("a")[-1].attrs["href"].split("=")[-1])

        for page_no in range(1, last_page_no):
            this_request_results = requests.get(page_url+str(page_no), headers=header, verify=False)
            this_soup = BeautifulSoup(this_request_results.text, "html.parser")
            this_tr = this_soup.findAll("table")[-2].findAll("tr")
            for i, tr in enumerate(this_tr):
                if i>1 and len(tr.findAll("td"))>2 and tr.findAll("td")[0] is not None:
                    df = df.append({"ymd" : ymd, "time" : tr.findAll("td")[0].findAll("span")[0].text.replace(":", ""), "price" : tr.findAll("td")[1].findAll("span")[0].text.replace(",","")}, ignore_index=True)

    now_date = datetime.datetime.now().strftime("%Y-%m-%d")
    save_file_name = "./outputs/{}_포스코 주식정보 수집 결과({} ~ {}).xlsx".format(now_date, start_ymd, end_ymd)
    df.to_excel(save_file_name, index=False)
    print("주식정보 수집을 완료하였습니다. 저장된 파일명 [{}]".format(save_file_name))
    display(df)

20220406 : 수집중...
주식정보 수집을 완료하였습니다. 저장된 파일명 [./outputs/2022-04-07_포스코 주식정보 수집 결과(20220406 ~ 20220406).xlsx]


,ymd,time,price
0,20220406,1555,285000
1,20220406,1549,285000
2,20220406,1547,285000
3,20220406,1546,285000
4,20220406,1545,285000
...,...,...,...
375,20220406,0914,286000
376,20220406,0913,286000
377,20220406,0912,285500
378,20220406,0911,285500
